In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_session = SparkSession.builder\
        .master("spark://localhost:7077") \
        .appName("Dinindu_Task-B")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/18 12:15:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# A JSON dataset is pointed to by path.
comments_json = spark_session.read.format('json').load("hdfs://130.238.28.245:9000/RC_2011-08")

In [6]:
comments_json.columns

['archived',
 'author',
 'author_flair_css_class',
 'author_flair_text',
 'body',
 'controversiality',
 'created_utc',
 'distinguished',
 'downs',
 'edited',
 'gilded',
 'id',
 'link_id',
 'name',
 'parent_id',
 'retrieved_on',
 'score',
 'score_hidden',
 'subreddit',
 'subreddit_id',
 'ups']

In [7]:
# Create an Spark SQL view for comments_json with the name "comments"
comments_json.createOrReplaceTempView("comments")

### To find the Top 10 authors who has the highest aggregate scores

In [8]:
spark_session.sql(
    "SELECT author, SUM(score) as total_score, COUNT(created_utc) as comments_count, AVG(score) as score_per_comment \
     FROM comments \
     WHERE author!= '[deleted]' \
     GROUP BY author \
     ORDER BY total_score DESC"
).show(25)

+--------------------+-----------+--------------+------------------+
|              author|total_score|comments_count| score_per_comment|
+--------------------+-----------+--------------+------------------+
|ProbablyHittingOnYou|      42822|          1128|37.962765957446805|
|         Prufrock451|      37309|           451|  82.7250554323725|
|        Jonnymagic00|      35767|           146| 244.9794520547945|
|           Warlizard|      27760|          2296|12.090592334494774|
|      LimblessWonder|      24167|           501|48.237524950099804|
|          AMfuckingA|      23105|           166| 139.1867469879518|
|              Skopak|      22744|           115|197.77391304347827|
|             Scarker|      20555|           622|33.046623794212216|
|              Copo55|      20216|           556| 36.35971223021583|
|             feureau|      19647|          1718|11.435972060535507|
|    ggggbabybabybaby|      19298|           739|26.113667117726656|
|       pseudolobster|      18027|

### To find the Top 10 authors who has written the most controversial comments

In [9]:
spark_session.sql(
    "SELECT author, SUM(controversiality) as total_controversiality, COUNT(created_utc) as comments_count \
     FROM comments \
     WHERE author!= '[deleted]' \
     GROUP BY author \
     ORDER BY total_controversiality DESC"
).show(25)

+----------------+----------------------+--------------+
|          author|total_controversiality|comments_count|
+----------------+----------------------+--------------+
|         simonbp|                     0|             2|
|          phi186|                     0|           186|
|       magistera|                     0|            21|
|        1950sGuy|                     0|           151|
|         catcard|                     0|            70|
|       tjsterc17|                     0|            94|
|       jdarp1984|                     0|            42|
|         cograve|                     0|            22|
|  SavageHunter77|                     0|             6|
|       bonzairob|                     0|            24|
|   dearsomething|                     0|           130|
|        madjecks|                     0|            51|
|      bijibijmak|                     0|            97|
|          gegtik|                     0|            66|
|         patssle|             

### Kill the application once you are done with the analysis :)

In [10]:
spark_session.stop()